In [104]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import json
from pandas import json_normalize
import re
from datetime import datetime
import csv

load_dotenv

token = os.getenv("API_TOKEN")

headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application.json'
}
response = requests.get('https://api.clashofclans.com/v1/clans/%232GLLPQYLL/currentwar', headers=headers)
response = response.json()

In [107]:
dt = datetime.now()
if response['state'] == 'inWar':
    with open(f'war{dt.month}.{dt.day}.{dt.year}time{dt.hour}.{dt.minute}.json', "a") as json_file:
        json.dump(response, json_file, indent=4)
elif response['state'] == 'warEnded':
    with open(f'war{dt.month}.{dt.day}.{dt.year}ended.json', "a") as json_file:
        json.dump(response, json_file, indent=4)

In [67]:
data = json_normalize(response, record_path=['clan', 'members'])
data

,tag,name,townhallLevel,mapPosition,attacks,opponentAttacks,bestOpponentAttack.attackerTag,bestOpponentAttack.defenderTag,bestOpponentAttack.stars,bestOpponentAttack.destructionPercentage,bestOpponentAttack.order,bestOpponentAttack.duration
0,#UGRL8LV8,mcnasty,13,4,"[{'attackerTag': '#UGRL8LV8', 'defenderTag': '...",1,#29PP8GCJ,#UGRL8LV8,3.0,100.0,14.0,79.0
1,#8Q8UQL8Y,NMbaseball22,11,7,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2,#8CRG2JR9,MeekMulli,12,5,"[{'attackerTag': '#8CRG2JR9', 'defenderTag': '...",0,NaN,NaN,NaN,NaN,NaN,NaN
3,#G222UU009,Ben Dover,11,9,"[{'attackerTag': '#G222UU009', 'defenderTag': ...",0,NaN,NaN,NaN,NaN,NaN,NaN
4,#2RR2PVRG,Alwayz High,15,2,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
5,#9PRCGP9Y,Luke,15,3,"[{'attackerTag': '#9PRCGP9Y', 'defenderTag': '...",1,#29PP8GCJ,#9PRCGP9Y,3.0,100.0,13.0,94.0
6,#9GGVU8C8,Shamrock 2.0,16,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
7,#GYQ09LYRR,winston,12,6,NaN,1,#L2UV99P0Y,#GYQ09LYRR,1.0,90.0,2.0,101.0
8,#GPC0JCLPG,Tony,10,14,"[{'attackerTag': '#GPC0JCLPG', 'defenderTag': ...",1,#99JVYVYR8,#GPC0JCLPG,2.0,85.0,4.0,106.0
9,#2VCU0JLL,CannonGannon10,11,8,NaN,1,#2LCQQL0RY,#2VCU0JLL,2.0,75.0,1.0,109.0


In [68]:
members_df = json_normalize(response, record_path=['clan', 'members'], meta=[])
total_members_df = members_df.copy()


In [69]:
attacks_df = json_normalize(
    members_df.to_dict(orient='records'),  # Convert DataFrame to dicts
    record_path='attacks',
    meta=['name'],
    meta_prefix='member_'
)
attacks_df

,attackerTag,defenderTag,stars,destructionPercentage,order,duration,member_name
0,#UGRL8LV8,#LCC2QPL0U,3,100,11,85,mcnasty
1,#UGRL8LV8,#8Y0GGCG2C,3,100,12,89,mcnasty
2,#8CRG2JR9,#Q2UU98LL,3,100,7,98,MeekMulli
3,#8CRG2JR9,#2LCQQL0RY,3,100,9,114,MeekMulli
4,#G222UU009,#GQ2LLRYQU,3,100,16,101,Ben Dover
5,#9PRCGP9Y,#9GVCUJPQ,3,100,15,98,Luke
6,#GPC0JCLPG,#JVQP0RUJ,1,57,5,120,Tony
7,#GPC0JCLPG,#GPVCL08RU,3,100,6,92,Tony
8,#GPRURPLRG,#99JVYVYR8,2,83,8,107,Wreckit Ralph


In [70]:
attacks_df['attack_num'] = attacks_df.groupby('member_name').cumcount() + 1
attacks_df



,attackerTag,defenderTag,stars,destructionPercentage,order,duration,member_name,attack_num
0,#UGRL8LV8,#LCC2QPL0U,3,100,11,85,mcnasty,1
1,#UGRL8LV8,#8Y0GGCG2C,3,100,12,89,mcnasty,2
2,#8CRG2JR9,#Q2UU98LL,3,100,7,98,MeekMulli,1
3,#8CRG2JR9,#2LCQQL0RY,3,100,9,114,MeekMulli,2
4,#G222UU009,#GQ2LLRYQU,3,100,16,101,Ben Dover,1
5,#9PRCGP9Y,#9GVCUJPQ,3,100,15,98,Luke,1
6,#GPC0JCLPG,#JVQP0RUJ,1,57,5,120,Tony,1
7,#GPC0JCLPG,#GPVCL08RU,3,100,6,92,Tony,2
8,#GPRURPLRG,#99JVYVYR8,2,83,8,107,Wreckit Ralph,1


In [71]:
attacks_df['total_attacks'] = attacks_df.groupby('member_name')['attack_num'].transform('max')
attacks_df

,attackerTag,defenderTag,stars,destructionPercentage,order,duration,member_name,attack_num,total_attacks
0,#UGRL8LV8,#LCC2QPL0U,3,100,11,85,mcnasty,1,2
1,#UGRL8LV8,#8Y0GGCG2C,3,100,12,89,mcnasty,2,2
2,#8CRG2JR9,#Q2UU98LL,3,100,7,98,MeekMulli,1,2
3,#8CRG2JR9,#2LCQQL0RY,3,100,9,114,MeekMulli,2,2
4,#G222UU009,#GQ2LLRYQU,3,100,16,101,Ben Dover,1,1
5,#9PRCGP9Y,#9GVCUJPQ,3,100,15,98,Luke,1,1
6,#GPC0JCLPG,#JVQP0RUJ,1,57,5,120,Tony,1,2
7,#GPC0JCLPG,#GPVCL08RU,3,100,6,92,Tony,2,2
8,#GPRURPLRG,#99JVYVYR8,2,83,8,107,Wreckit Ralph,1,1


In [72]:
total_attacks_df = attacks_df[['member_name', 'total_attacks']].drop_duplicates().set_index('member_name')
total_attacks_df

,total_attacks
member_name,
mcnasty,2
MeekMulli,2
Ben Dover,1
Luke,1
Tony,2
Wreckit Ralph,1


In [73]:
pivot_df = attacks_df.pivot_table(
    index=['member_name'], 
    columns='attack_num', 
    values=['attackerTag', 'defenderTag', 'stars', 'destructionPercentage', 'order', 'duration'], 
    aggfunc='first'
)
pivot_df

attackerTag             defenderTag              \
attack_num              1           2           1           2   
member_name                                                     
Ben Dover      #G222UU009         NaN  #GQ2LLRYQU         NaN   
Luke            #9PRCGP9Y         NaN   #9GVCUJPQ         NaN   
MeekMulli       #8CRG2JR9   #8CRG2JR9   #Q2UU98LL  #2LCQQL0RY   
Tony           #GPC0JCLPG  #GPC0JCLPG   #JVQP0RUJ  #GPVCL08RU   
Wreckit Ralph  #GPRURPLRG         NaN  #99JVYVYR8         NaN   
mcnasty         #UGRL8LV8   #UGRL8LV8  #LCC2QPL0U  #8Y0GGCG2C   

              destructionPercentage        duration        order       stars  \
attack_num                        1      2        1      2     1     2     1   
member_name                                                                    
Ben Dover                     100.0    NaN    101.0    NaN  16.0   NaN   3.0   
Luke                          100.0    NaN     98.0    NaN  15.0   NaN   3.0   
MeekMulli                     100.0  100.0     98.0  114.0   7.0   9.0   3.0   
Tony                           57.0  100.0    120.0   92.0   5.0   6.0   1.0   
Wreckit Ralph                  83.0    NaN    107.0    NaN   8.0   NaN   2.0   
mcnasty                       100.0  100.0     85.0   89.0  11.0  12.0   3.0   

                    
attack_num       2  
member_name         
Ben Dover      NaN  
Luke           NaN  
MeekMulli      3.0  
Tony           3.0  
Wreckit Ralph  NaN  
mcnasty        3.0

In [74]:
pivot_df.columns = [f"{col[0]}_{col[1]}" for col in pivot_df.columns]

In [75]:
def extract_text_and_number(column_name):
    match = re.match(r'(.+?)_(\d+)', column_name)
    return (int(match.group(2)), match.group(1)) if match else (float('inf'), column_name)


In [76]:
sorted_columns = sorted(pivot_df.columns, key=lambda x: extract_text_and_number(x))
sorted_columns

['attackerTag_1',
 'defenderTag_1',
 'destructionPercentage_1',
 'duration_1',
 'order_1',
 'stars_1',
 'attackerTag_2',
 'defenderTag_2',
 'destructionPercentage_2',
 'duration_2',
 'order_2',
 'stars_2']

In [77]:
pivot_df = pivot_df[sorted_columns]

In [78]:
final_df = pivot_df.join(total_attacks_df)
final_df

,attackerTag_1,defenderTag_1,destructionPercentage_1,duration_1,order_1,stars_1,attackerTag_2,defenderTag_2,destructionPercentage_2,duration_2,order_2,stars_2,total_attacks
member_name,,,,,,,,,,,,,
Ben Dover,#G222UU009,#GQ2LLRYQU,100.0,101.0,16.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1
Luke,#9PRCGP9Y,#9GVCUJPQ,100.0,98.0,15.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1
MeekMulli,#8CRG2JR9,#Q2UU98LL,100.0,98.0,7.0,3.0,#8CRG2JR9,#2LCQQL0RY,100.0,114.0,9.0,3.0,2
Tony,#GPC0JCLPG,#JVQP0RUJ,57.0,120.0,5.0,1.0,#GPC0JCLPG,#GPVCL08RU,100.0,92.0,6.0,3.0,2
Wreckit Ralph,#GPRURPLRG,#99JVYVYR8,83.0,107.0,8.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1
mcnasty,#UGRL8LV8,#LCC2QPL0U,100.0,85.0,11.0,3.0,#UGRL8LV8,#8Y0GGCG2C,100.0,89.0,12.0,3.0,2


In [79]:
new_name_df = final_df
new_name_df['Avg %'] = new_name_df[['destructionPercentage_1', 'destructionPercentage_2']].mean(axis=1)
new_name_df['Total Stars'] = new_name_df[["stars_1", "stars_2"]].sum(axis=1)

In [80]:
new_order = ['total_attacks', 'Avg %', 'destructionPercentage_1', 'destructionPercentage_2', 
             'stars_1', 'stars_2', 'Total Stars']
new_name_df = new_name_df[new_order]
final_name_df = new_name_df.rename(columns={'destructionPercentage_1' : 'Attack 1 %', 'destructionPercentage_2' : 'Attack 2 %', 
                                            'stars_1' : 'Attack 1 Stars', 'stars_2' : 'Attack 2 Stars', 'total_attacks' : 'Total Attacks'})
final_name_df

,Total Attacks,Avg %,Attack 1 %,Attack 2 %,Attack 1 Stars,Attack 2 Stars,Total Stars
member_name,,,,,,,
Ben Dover,1,100.0,100.0,NaN,3.0,NaN,3.0
Luke,1,100.0,100.0,NaN,3.0,NaN,3.0
MeekMulli,2,100.0,100.0,100.0,3.0,3.0,6.0
Tony,2,78.5,57.0,100.0,1.0,3.0,4.0
Wreckit Ralph,1,83.0,83.0,NaN,2.0,NaN,2.0
mcnasty,2,100.0,100.0,100.0,3.0,3.0,6.0


In [81]:
try:
    total_members_df = total_members_df.set_index('name')
except KeyError:
    pass


In [82]:
final_name_df["Total Clan Stars"] = final_name_df['Total Stars'].sum()
final_name_df

,Total Attacks,Avg %,Attack 1 %,Attack 2 %,Attack 1 Stars,Attack 2 Stars,Total Stars,Total Clan Stars
member_name,,,,,,,,
Ben Dover,1,100.0,100.0,NaN,3.0,NaN,3.0,24.0
Luke,1,100.0,100.0,NaN,3.0,NaN,3.0,24.0
MeekMulli,2,100.0,100.0,100.0,3.0,3.0,6.0,24.0
Tony,2,78.5,57.0,100.0,1.0,3.0,4.0,24.0
Wreckit Ralph,1,83.0,83.0,NaN,2.0,NaN,2.0,24.0
mcnasty,2,100.0,100.0,100.0,3.0,3.0,6.0,24.0


In [83]:
combined_df = final_name_df.reindex(total_members_df.index)
combined_df.fillna(0, inplace=True)
combined_df.sort_values(["Total Attacks", 'Total Stars', 'Avg %'], ascending=False, inplace=True)

In [84]:
def colors(row):
    if row['Total Attacks'] == 2.0:
        color = "rgb(0,255,0)"
    elif row['Total Attacks'] == 1.0:
        color = "yellow"
    else:
        color = "red"
    return [f'color: {color}'] * len(row) 

In [85]:
styled_df = combined_df.style.apply(colors, axis=1).format('{:.1f}')
styled_df


,Total Attacks,Avg %,Attack 1 %,Attack 2 %,Attack 1 Stars,Attack 2 Stars,Total Stars,Total Clan Stars
name,,,,,,,,
mcnasty,2.0,100.0,100.0,100.0,3.0,3.0,6.0,24.0
MeekMulli,2.0,100.0,100.0,100.0,3.0,3.0,6.0,24.0
Tony,2.0,78.5,57.0,100.0,1.0,3.0,4.0,24.0
Ben Dover,1.0,100.0,100.0,0.0,3.0,0.0,3.0,24.0
Luke,1.0,100.0,100.0,0.0,3.0,0.0,3.0,24.0
Wreckit Ralph,1.0,83.0,83.0,0.0,2.0,0.0,2.0,24.0
NMbaseball22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alwayz High,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Shamrock 2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
